In [2]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [3]:
with open('interview.json') as file:
    data = json.load(file)

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [6]:
enc = LabelEncoder()
enc.fit(training_labels)
training_labels = enc.transform(training_labels)

In [7]:
vocab_size = 10000
embedding_dim = 16
max_len = 20
trunc_type = 'post'
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, truncating=trunc_type, maxlen=max_len)

In [8]:
classes = len(labels)

In [9]:
model = tf.keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(classes, activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 68        
Total params: 160,612
Trainable params: 160,612
Non-trainable params: 0
_________________________________________________________________


In [11]:
training_labels_final = np.array(training_labels)

In [12]:
EPOCHS = 500
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(padded, training_labels_final, epochs=EPOCHS)


Train on 27 samples
Epoch 1/500
27/27 [==============================] - 1s 40ms/sample - loss: 1.3934 - accuracy: 0.1481
Epoch 2/500
27/27 [==============================] - 0s 148us/sample - loss: 1.3914 - accuracy: 0.1481
Epoch 3/500
27/27 [==============================] - 0s 111us/sample - loss: 1.3894 - accuracy: 0.1481
Epoch 4/500
27/27 [==============================] - 0s 185us/sample - loss: 1.3877 - accuracy: 0.1481
Epoch 5/500
27/27 [==============================] - 0s 445us/sample - loss: 1.3869 - accuracy: 0.1481
Epoch 6/500
27/27 [==============================] - 0s 148us/sample - loss: 1.3858 - accuracy: 0.1481
Epoch 7/500
27/27 [==============================] - 0s 111us/sample - loss: 1.3846 - accuracy: 0.4444
Epoch 8/500
27/27 [==============================] - 0s 148us/sample - loss: 1.3833 - accuracy: 0.4444
Epoch 9/500
27/27 [==============================] - 0s 148us/sample - loss: 1.3822 - accuracy: 0.4444
Epoch 10/500
27/27 [==============================] - 

In [13]:
def check():
    print('Speecial_AI_Bot_v1')
    while True:
        string = input('Enter: ')
        if string == 'quit': break
        result = model.predict(pad_sequences(tokenizer.texts_to_sequences([string]),
                                             truncating=trunc_type, maxlen=max_len))[0]

        category = enc.inverse_transform([np.argmax(result)]) # labels[np.argmax(result)]
        
        results_index = np.argmax(result)
        if result[results_index] > 0.4:   
            for i in data['intents']:
                if i['tag']==category:
                    print(np.random.choice(i['responses']), results_index)
        else:
            print("Sorununuz anlaşılmamıştır Lütfen tekrarlayınız")


In [ ]:
check()

Speecial_AI_Bot_v1


Enter:  Selam


Selam 2


Enter:  Benim bazı sağlık sorunlarım vardı


Size Nasıl Yardımcı Olabilirim 2


Enter:  Son bir kaç gündür baş ağrısı çekiyorum hastaneye gitmeme sebep olacak bir belirtmidir ?


Hastaneye Gitmenize Gerek Yoktur 1


Enter:  uzun süredir geçmeyen ishal hastaneye gitmeme sebep olurmu


Hastaneye Gelmeden önce endoskopi yaptırınız 0


In [56]:
tf.keras.models.save_model(model, "z_bot")

INFO:tensorflow:Assets written to: z_bot\assets
